In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
#df= pd.read_csv('../../data/more_features/with_correlated_feature_0.1.csv')
df =pd.read_csv('../../data/more_features/more_feats_correlated_noisy_4_missing_1.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,8.725667,-1.000000,2.344526,-1.000000,-1.000000,96.678418,-1.000000,2.707637,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,26.177001,-1.000000,Hemolytic anemia
1,12.430232,231.691743,5.143660,5.339158,484.590922,87.353532,-1.000000,4.268940,0,-1.000000,-1.000000,-1.000000,8.718062,-1.000000,-1.000000,37.290695,-1.000000,No anemia
2,6.737760,360.843434,-1.000000,5.481140,421.177091,75.086568,82.986344,2.691997,0,0.458698,19.596767,69.527433,-1.000000,-1.000000,-1.000000,20.213279,19.703433,Anemia of chronic disease
3,12.313034,-1.000000,2.765581,0.000000,-1.000000,99.494524,-1.000000,3.712677,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.939103,-1.000000,Unspecified anemia
4,6.758140,-1.000000,1.696525,-1.000000,-1.000000,86.643128,199.741346,2.339992,0,0.887158,80.614718,109.197814,18.770122,7.668335,63.159054,20.274421,-1.000000,Aplastic anemia


In [4]:
utils.get_dt_performance(df)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

(0.6365714285714286,
 0.6190348343275895,
 0.7819538474170686,
 datetime.timedelta(microseconds=7239))

In [5]:
df.label.value_counts()

No anemia                               16000
Anemia of chronic disease                8816
Iron deficiency anemia                   8309
Aplastic anemia                          8157
Unspecified anemia                       8132
Vitamin B12/Folate deficiency anemia     8124
Hemolytic anemia                         8057
Inconclusive diagnosis                   4405
Name: label, dtype: int64

In [6]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional in

((56000, 17), (14000, 17), (56000,), (14000,))

In [7]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [ ]:
for steps in [int(5e6), int(5.5e6), int(6e6), int(6.5e6), int(7e6), int(7.5e6)]:
#for steps in [int(2e3)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_3_11000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,3.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, Aplastic anemia]",6.0,6.0
1,3.0,1.0,0.0,-1.0,0.0,"[hemoglobin, mcv, Vitamin B12/Folate deficienc...",2.0,1.0
2,3.0,2.0,0.0,-1.0,0.0,"[hemoglobin, mcv, Vitamin B12/Folate deficienc...",2.0,1.0
3,4.0,3.0,0.0,-1.0,0.0,"[hemoglobin, mcv, gender, Aplastic anemia]",5.0,6.0
4,3.0,4.0,0.0,-1.0,0.0,"[hemoglobin, mcv, Aplastic anemia]",5.0,6.0


In [15]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [16]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [10]:
test_df.y_pred.value_counts()

7.0    3606
6.0    2625
3.0    2613
1.0    2206
0.0    2128
2.0     527
4.0     264
5.0      31
Name: y_pred, dtype: int64

In [11]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

43.40714285714286

In [12]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(3.158714285714286, -0.17557142857142857)

In [13]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.43407142857142855, 0.3753685428004125, 0.6695926627226381)

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)